#### Introduction

SQL (Structured Query Langugage) is the language we use to work with a RDB (relational database). Relational databases are the backbone of many software systems, and offer us the ability to safely store and quickly access large amounts of data.

##### How to interface with database?
 - "Raw" SQL inside database console
 - Application layer (Python, Node, etc.)

##### Source data
 
**[2018 - 2019 Arts Data Survey](https://data.cityofnewyork.us/Education/2018-2019-Arts-Data-Survey/5cxm-c27f)**

##### Resources
**[Google Dataset Search](https://datasetsearch.research.google.com/)**

**[PostgreSQL Exercises](https://pgexercises.com/index.html)**

---

##### Complete sign-in form in STUDENT account:
```
> Login as Student > Desktop chrome icon > Complete form
``` 

##### Switch to CSCRESEARCHER account:
```
> Start menu > Person icon > CSC Researcher
```

##### Open pgAdmin4:
```
> Search "pgAdmin4" in search bar > Open
```

##### Create a database:
```
> Double click Servers > Right-click Databases > create > Database...
> Database name: art_survey_db
```

##### Create a table:
```
> Schemas > Public > (right-click) Tables > Create > Table... > Columns > +
Columns:
    id: integer (no bracket) 
    name: text (no bracket) 
    age: integer (no bracket) 
```        

##### Show Query Editor:
```
> Tools > Query Editor
```

##### Add data:
```
INSERT INTO [schema].[table] VALUES ([column_name], [column_name], [column_name]) VALUES ([value], [value], [value])

Note: Replace brackets & identifiers with values
```

##### Queries:
```
Execute query with F5 or Bolt icon

INSERT INTO public.main (id, name, age) VALUES (1, 'Mary', 30)
INSERT INTO public.main (id, name, age) VALUES (2, 'Sam', 50)
INSERT INTO public.main (id, name, age) VALUES (3, 'Diana', 25)
```

In [ ]:
import requests
import pandas as pd
import requests
import warnings
import os
warnings.filterwarnings('ignore')
root_url = "https://github.com/Barnard-Computational-Science-Center/data-engineering-fall20/blob/master"


PASSWORD = ''
DB = ''
USER = 'postgres'
HOST = 'localhost'
PORT = '5432'


def connect(USER, PASSWORD, DB, HOST, PORT):
    import sqlalchemy
    import psycopg2
    url = f'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}'
    engine = sqlalchemy.create_engine(url, client_encoding='utf8')
    return engine

engine = connect(USER, PASSWORD, DB, HOST, PORT)
engine

In [ ]:
query = 'create table public.test (column_1 int);'
engine.execute(query)

In [ ]:
query = 'INSERT INTO public.test (column_1) VALUES (3)'
engine.execute(query)

In [ ]:
query = 'SELECT * FROM public.test'
engine.execute(query).fetchall()

In [ ]:
###################
##### EXTRACT #####
###################

#arts_report = "arts_report.xlsx"
arts_report_url = os.path.join(root_url, 'arts_report.xlsx?raw=true')
art_df = pd.read_excel(arts_report_url)

cols = list(art_df)[0:17]
art_df = art_df[cols]

len(art_df)
art_df.head()

In [ ]:
#ms_endpoint = "https://data.cityofnewyork.us/resource/fdpt-nzk4.json"
#ms_df = pd.read_json(ms_endpoint)

#ms_data = "middle_schools.csv"
ms_data_url = os.path.join(root_url, 'middle_schools.csv?raw=true')
ms_df = pd.read_csv(ms_data_url, usecols = ['schooldbn', 'name', 'address'])

In [ ]:
########################
##### School Table #####
########################

```
New table: school

id
schooldbn
name
address

-----

create table public.school
(
	id serial not null
		constraint table_name_pk
			primary key,
	schooldbn varchar,
	name varchar,
	address varchar
);
```

In [ ]:
ids = list(range(1, len(ms_df) + 1)) # Assign unique schools Ids
ms_df['id'] = ids

ms_df = ms_df[['id','schooldbn', 'name', 'address']]
ms_df.head()

In [ ]:
ms_df.to_sql('school', engine, index = False, if_exists = 'append')
print("finished.")

In [ ]:
schools = pd.read_sql_table('school', engine)
schools.head()

In [ ]:
########################
##### Person Table #####
########################

In [ ]:
# Filter values
filter_list = ms_df['schooldbn']
art_df = art_df[art_df['Q0_DBN'].isin(filter_list)]
len(art_df)

In [ ]:
school_mapper = dict(ms_df[['schooldbn', 'id']].values)
art_df['Q0_DBN'] = art_df['Q0_DBN'].map(school_mapper)
art_df.head()

In [ ]:
renamed_columns = {
    "Q0_DBN": "school_id", 
    "Q1_1": "first_name", 
    "Q1_2": "last_name",
    "Q1_3": "email"
}

art_df.rename(columns = renamed_columns, inplace = True)
art_df = art_df.dropna(subset = ['first_name', 'last_name', 'email'], how = 'any')
art_df.head()

```
New Table: person

id
school_id --> school(id)
first_name
last_name
email

-----

create table person
(
	id serial not null
		constraint person_pk
			primary key,

    school_id int not null,
	first_name varchar not null,
	last_name varchar not null,
	email varchar not null,

	foreign key (school_id) references school(id)
);
```

In [ ]:
person_columns = ['school_id', 'first_name', 'last_name', 'email']
person_df = art_df[person_columns]
ids = list(range(1, len(person_df) + 1))
person_df['id'] = ids
person_df = person_df[['id','school_id', 'first_name', 'last_name', 'email']]
person_df.head()

In [ ]:
person_df.to_sql('person', engine, index = False, if_exists = 'append')
print("finished.")

In [ ]:
query = "INSERT INTO public.person (first_name, last_name, email) VALUES ('ted', 'smith', 'tsmith@gmail.com')"
engine.execute(query)

In [ ]:
############################
#####  Question Table ######
############################

```
Example schema: 
https://stackoverflow.com/questions/1764435/database-design-for-a-survey

New Table: question

id
question_name
question_type
question

-----

create table question

(
	id serial not null
		constraint question_pk
			primary key,

    question_name varchar not null,
    question_type varchar not null,
	question varchar not null

);
```

In [ ]:
question_url = os.path.join(root_url, "question.json?raw=true")
question_df = pd.read_json(question_url)
ids = list(range(1, len(question_df) + 1))
question_df['id'] = ids
question_columns = ['id', 'question_name', 'question_type', 'question']
question_df = question_df[question_columns]
question_df.head()

In [ ]:
question_df.to_sql('question', engine, index = False, if_exists = 'append')
print("finished.")

In [ ]:
############################
#####  Response Table ######
############################

```
New Table: response

id
person_id --> person(id)
question_id --> question(id)
response

-----

create table response

(
	id serial not null
		constraint response_pk
			primary key,

    person_id int not null,
    question_id int not null,
	response varchar not null,
    
    foreign key (person_id) references person(id),
    foreign key (question_id) references question(id)

);
```

In [ ]:
art_df.head()

In [ ]:
drop_columns = ['school_id','first_name', 'last_name']
art_df.drop(drop_columns, axis = 1, inplace = True)

In [ ]:
person_mapper = dict(person_df[['email', 'id']].values)
art_df['email'] = art_df['email'].map(person_mapper)
art_df.rename(columns = {'email': 'person_id'}, inplace = True)

In [ ]:
art_df.head()

```
 What should the data look like in a relational database? What about Python?
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

{"id": 1, "person_id": 1, "question_id": 1, "response": "0"},
{"id": 2, "person_id": 1, "question_id": 2, "response": "0"},
{"id": 3, "person_id": 1, "question_id": 3, "response": "1"},
{"id": 4, "person_id": 1, "question_id": 4, "response": "0"},
 ....
 ...
 ..
 .
 ..
 ...
 ....
{"id": 433, "person_id": 1, "question_id": 10, "response": "0"},
{"id": 433, "person_id": 1, "question_id": 11, "response": "0"},
{"id": 433, "person_id": 1, "question_id": 12, "response": "1"},
{"id": 433, "person_id": 1, "question_id": 13, "response": "0"}
```

In [ ]:
response_df = art_df.copy()
response_melted = response_df.melt(id_vars = ['person_id'], var_name ='question_id', value_name = 'response')
response_melted.sort_values(by=['person_id', 'question_id'], inplace = True)
#response_melted.loc[(response_melted['person_id'] == 5) & (response_melted['variable'] == 2)] # check to see if it worked.

In [ ]:
response_melted.head()

In [ ]:
response_melted.tail()

In [ ]:
response_mapper = dict(question_df[['question_name', 'id']].values)
response_melted['question_id'] = response_melted['question_id'].map(response_mapper)

In [ ]:
response_melted

In [ ]:
ids = list(range(1, len(response_melted) + 1))
response_melted['id'] = ids
response_melted = response_melted[['id', 'person_id','question_id','response']]

In [ ]:
response_melted

In [ ]:
response_melted.to_sql('response', engine, index = False, if_exists = 'append')
print("finished.")